In [4]:
import pandas as pd
import yfinance as yf
import numpy as np
import warnings
import random
from pathlib import Path
import pickle as pkl
from main_utility_functions import ticker_gen

warnings.filterwarnings("ignore")

In [10]:
'''PART 1:DOWNLOADING DATA'''

with open('univs.pkl','rb') as f:  #rb is read binary code
     univs= pkl.load(f)

# Date range
start = '2003-11-01'
end = '2022-11-01'

# Tickers of assets, note that univs contains 10 universes!
assets = list(set(sum(univs, []))) #change this number to switch universes

#THIS STEP IS IMPT!
#yfinance will download tickers in sorted order,regardless of input order
#this step sorts the column names, if not the labelling will be wrong
assets.sort()
# Downloading data
prices = yf.download(assets, start=start, end=end)
prices = prices.dropna()
#keep adjusted close prices only
assets_prices=prices.loc[:, ('Adj Close', slice(None))]
assets_prices.columns = assets

[*********************100%%**********************]  18 of 18 completed


In [12]:
# Calculate assets returns
returns = assets_prices.pct_change()#return on day 2 is day1-day2 comparison

# Selecting Dates for Rebalancing

#index contains datetime indexes of first day of each month
index = returns.groupby([returns.index.year, returns.index.month]).head(1).index

#index_ is the integer version of index, in addition to:
#only after cov_period will weights be calculated, so index_ also accounts for this
cov_period=252 #n.o. of days in one trading year
index_ = [returns.index.get_loc(x) for x in index if returns.index.get_loc(x) >cov_period]

path = 'data_all.pkl'

if Path(path).exists():
    raise Exception('data.pkl already exists. Please delete it before running this script again.')
#save impt variables as a list, written in binary code
with open(path,'wb') as f:
    pkl.dump([cov_period, index_, returns],f)
